In [69]:
from bs4 import BeautifulSoup
import urllib 
import requests
import os
import pandas as pd
import cv2
from PIL import Image, ImageSequence
import shutil

In [35]:
def download_img(soup, imageStr, imageNum):
    for i in range(len(soup)):
        imgURL = soup[i].find("img")["src"].replace('//', '')
#         print(i)
        print(imgURL)
        name = imgURL.split('/')[-1]
#         print(name)
        
        if 'www' in imgURL:
            imgURL= imgURL[imgURL.index('www'):]
        if not 'https' in imgURL:
            imgURL = 'https://'+imgURL
            
        urllib.request.urlretrieve(imgURL, os.path.join(imageStr, str(imageNum)+'_'+name))

In [36]:
url = "https://www.chuu.co.kr/product/list.html?cate_no=2529"
url = url + '&page='+str(1)
fp = urllib.request.urlopen(url)
source = fp.read();
fp.close()

soup = BeautifulSoup(source, 'html.parser')

soup1= soup.findAll("li", class_='xans-element- xans-product xans-product-displaycategory xans-record-')

dicts = dict()

for s in soup1:
    soups = s.findAll('a')
    tmp =dict()
    for ss in soups:
        if len(ss)==1:
            code=ss['href'].split('cate_no=')[-1]
            name = ss.get_text().replace(' ()', '')
            cate = name+'_'+code
        
        else:
            code=ss['href'].split('cate_no=')[-1]
            name = ss.get_text().replace(' ()', '')
            tmp[name]=code
    dicts[cate] = tmp

print(dicts)

{'MADE_25': {}, 'TOP_2478': {'tee': '2529', 'blouse': '2527', 'knit': '2530', 'sleeveless': '2528', 'bra top': '2838'}, 'BOTTOM_2486': {'PANTS': '2483', 'SKIRT': '2482'}, 'DRESS_2480': {}, 'OUTER_2481': {}, 'LINGERIE_2485': {'LINGERIE': '2491', 'SECRET BODY': '2492', 'PAJAMA': '2493'}, 'ACC_2484': {'JEWELRY': '2487', 'SHOES & BAG': '2488', 'SOCKS': '2489', 'ETC': '2490'}, 'BIKINI_2864': {}}


In [37]:
def crowling(cate, code, cate_sub=None):
    url = "https://www.chuu.co.kr/product/list.html?cate_no={}".format(code)
    fp = urllib.request.urlopen(url)
    source = fp.read();
    fp.close()

    soup = BeautifulSoup(source, 'html.parser')

    #접근할 페이지 번호
    pageNum = 1
    soup= soup.findAll("div", class_='xans-element- xans-product xans-product-normalpaging')
    end_pageNum= len(soup[0].findAll("li", class_='xans-record-'))

    # end_pageNum = 5
    #저장할 이미지 경로 및 이름 (data폴더에 face0.jpg 형식으로 저장)
    imageNum = 0
    if cate_sub==None:
        imageStr = "data/{}".format(cate)
    else:
        imageStr = "data/{}/{}".format(cate, cate_sub)
    print('save path :', imageStr) 
    
    if not os.path.isdir(imageStr):
        os.makedirs(imageStr)
        
    while pageNum < end_pageNum+1:
        url_p = url + '&page='+str(pageNum)
        print(url_p)
        fp = urllib.request.urlopen(url_p)
        source = fp.read();
        fp.close()

        soup = BeautifulSoup(source, 'html.parser')
        soup = soup.findAll("div", class_='box')


        #######
        ### download_img
        #######

        for i in range(len(soup)):
            imgURL = soup[i].find("img")["src"].replace('//', '')
    #         print(i)
    #         print(imgURL)
            name = imgURL.split('/')[-1]
    #         print(name)

            if 'txt_progress' in imgURL:
                continue

            if 'www' in imgURL:
                imgURL= imgURL[imgURL.index('www'):]
            if not 'https' in imgURL:
                imgURL = 'https://'+imgURL
            if not '://' in imgURL:
                imgURL = imgURL.replace(':', '://')
            
            print(imgURL)
            urllib.request.urlretrieve(imgURL, os.path.join(imageStr, str(imageNum)+'_'+name))
            imageNum+=1
        pageNum+=1


In [38]:
for cate in dicts.keys():
#     print(cate)
#     print(dicts[cate].keys())
    
    if list(dicts[cate].keys())==[]:
#         print(cate)
        cate_name = cate.split('_')[0]
        code = cate.split('_')[-1]
        print(cate_name, code)
        crowling(cate_name, code)
    else:
#         print(cate)
        cate_name = cate.split('_')[0]
        for cate_sub in dicts[cate].keys():
            code = dicts[cate][cate_sub]
            print(cate_name, cate_sub, code)
            crowling(cate_name, code, cate_sub)
            

MADE 25
save path : data/MADE
https://www.chuu.co.kr/product/list.html?cate_no=25&page=1
https://www.chuu.co.kr/web/product/small/202103/dc862bfbf9241b1879e5c65ee01ee1a7.gif
https://www.chuu.co.kr/web/product/small/202109/4dad0c6ef9df7e1f9b6584c5bd996ccc.gif
https://www.chuu.co.kr/web/product/small/202109/31bac1e77a3cecae770e9c70eb91142f.gif
https://www.chuu.co.kr/web/product/small/202109/3a8d58baeb3cc439edad14a8e1df3737.gif
https://www.chuu.co.kr/web/product/small/202107/282247226de449e9ac66f9edb492a25d.gif
https://www.chuu.co.kr/web/product/small/202102/85fff7ffbb93b37bb805366804e65fb3.gif
https://www.chuu.co.kr/web/product/small/202109/3b6129e2c67956bc2283fda888a850a7.gif
https://www.chuu.co.kr/web/product/small/202109/7748a07198c1481797439e93001c5dad.gif
https://www.chuu.co.kr/web/product/small/202102/1965363cf6173f22bc30e9eb15f900aa.gif
https://www.chuu.co.kr/web/product/small/202108/214a15d495bcde11877908c0d941aa6e.gif
https://www.chuu.co.kr/web/product/small/202106/d42c65b5fce2b

In [80]:
import pandas as pd
root_dir = "./data/"

cnt = list()
total = 0
for (root, dirs, files) in os.walk(root_dir):
    print("# root : " + root)
    
    files = [i for i in files if not 'csv' in i]
    if len(files) > 0:
#         print("file: ",  len(files))
        cnt.append([root, len(files)])
        total+=len(files)
        
        for file in files:
#             print(os.path.join(root,file))
            
            im = Image.open(os.path.join(root,file))
            name = file.split('.')[0]
            if not os.path.isdir(root.replace('data', 'data_bmp2')):
                os.makedirs(root.replace('data', 'data_bmp2'))
            index = 1
            for frame in ImageSequence.Iterator(im):
                frame.save(root.replace('data', 'data_bmp2')+'/'+name+"_frame%d.bmp" % index)
                index += 1


# root : ./data/
# root : ./data/ACC
# root : ./data/ACC\ETC
# root : ./data/ACC\JEWELRY
# root : ./data/ACC\SHOES & BAG
# root : ./data/ACC\SOCKS
# root : ./data/BIKINI
# root : ./data/BOTTOM
# root : ./data/BOTTOM\PANTS
# root : ./data/BOTTOM\SKIRT
# root : ./data/DRESS
# root : ./data/LINGERIE
# root : ./data/LINGERIE\LINGERIE
# root : ./data/LINGERIE\PAJAMA
# root : ./data/LINGERIE\SECRET BODY
# root : ./data/MADE
# root : ./data/OUTER
# root : ./data/TOP
# root : ./data/TOP\blouse
# root : ./data/TOP\bra top
# root : ./data/TOP\knit
# root : ./data/TOP\sleeveless
# root : ./data/TOP\tee


In [81]:
import pandas as pd
root_dir = "./data_bmp2/"

cnt = list()
total = 0
for (root, dirs, files) in os.walk(root_dir):
    print("# root : " + root)
    
    files = [i for i in files if not 'csv' in i]
    if len(files) > 0:
#         print("file: ",  len(files))
        cnt.append([root, len(files)])
        total+=len(files)
    else:
        cnt.append([root, 0])
#         index = 1
#         for file in files:
# #             print(os.path.join(root,file))
            
#             im = Image.open(os.path.join(root,file))
#             name = file.split('.')[0]
#             if not os.path.isdir(root.replace('data', 'data_bmp')):
#                 os.makedirs(root.replace('data', 'data_bmp'))
                
#             for frame in ImageSequence.Iterator(im):
#                 frame.save(root.replace('data', 'data_bmp')+'/'+name+"_frame%d.bmp" % index)
#                 index += 1

cnt.append(['total', total])
df = pd.DataFrame(cnt, columns=['root', 'count'])
print(df)
df.to_csv('./data_bmp2/info.csv', index=False)

# root : ./data_bmp2/
# root : ./data_bmp2/ACC
# root : ./data_bmp2/ACC\ETC
# root : ./data_bmp2/ACC\JEWELRY
# root : ./data_bmp2/ACC\SHOES & BAG
# root : ./data_bmp2/ACC\SOCKS
# root : ./data_bmp2/BIKINI
# root : ./data_bmp2/BOTTOM
# root : ./data_bmp2/BOTTOM\PANTS
# root : ./data_bmp2/BOTTOM\SKIRT
# root : ./data_bmp2/DRESS
# root : ./data_bmp2/LINGERIE
# root : ./data_bmp2/LINGERIE\LINGERIE
# root : ./data_bmp2/LINGERIE\PAJAMA
# root : ./data_bmp2/LINGERIE\SECRET BODY
# root : ./data_bmp2/MADE
# root : ./data_bmp2/OUTER
# root : ./data_bmp2/TOP
# root : ./data_bmp2/TOP\blouse
# root : ./data_bmp2/TOP\bra top
# root : ./data_bmp2/TOP\knit
# root : ./data_bmp2/TOP\sleeveless
# root : ./data_bmp2/TOP\tee
                                root  count
0                       ./data_bmp2/      0
1                    ./data_bmp2/ACC      0
2                ./data_bmp2/ACC\ETC    255
3            ./data_bmp2/ACC\JEWELRY    189
4        ./data_bmp2/ACC\SHOES & BAG    631
5              ./data_

In [70]:


root_dir = "./data_bmp/"

cnt = list()
for (root, dirs, files) in os.walk(root_dir):
    print("# root : " + root)
    
    if not os.path.isdir(root.replace('data_bmp', 'data_bmp_sim')):
        os.makedirs(root.replace('data_bmp', 'data_bmp_sim'))
            
    files = [i for i in files if not 'csv' in i]
    if len(files) > 0:
        for f, file in enumerate(files):
            print(os.path.join(root,file))
            if f==0:
                img = cv2.imread(os.path.join(root,file))
                name_pre = file.split('_')[1]
                img_pre = img
            else:
                img = cv2.imread(os.path.join(root,file))
                name = file.split('_')[1]
                
                
                if name==name_pre:
#                 print(np.mean(img)-np.mean(img_pre))
#                 if np.array_equal(img, img_pre):
                    print(cv2.matchTemplate(img,img_pre,eval('cv2.TM_CCOEFF'))[0][0])
                    if cv2.matchTemplate(img,img_pre,eval('cv2.TM_CCOEFF'))[0][0]>1e8:
                        shutil.move(os.path.join(root,file), os.path.join(root.replace('data_bmp', 'data_bmp_sim'),file))
#                     else:
#                         img_pre=img
                img_pre=img
            
                name_pre = file.split('_')[1]
#             im = Image.open(os.path.join(root,file))
            
#             name = file.split('.')[0]
#             if not os.path.isdir(root.replace('data', 'data_bmp')):
#                 os.makedirs(root.replace('data', 'data_bmp'))
                
#             for frame in ImageSequence.Iterator(im):
#                 frame.save(root.replace('data', 'data_bmp')+'/'+name+"_frame%d.bmp" % index)
#                 index += 1

# root : ./data_bmp/
# root : ./data_bmp/ACC
# root : ./data_bmp/ACC\ETC
./data_bmp/ACC\ETC\0_b43c93c42a2b5655bf534e25776caaaa_frame1.bmp
./data_bmp/ACC\ETC\0_b43c93c42a2b5655bf534e25776caaaa_frame2.bmp
-287264100.0
./data_bmp/ACC\ETC\0_b43c93c42a2b5655bf534e25776caaaa_frame3.bmp
-118301470.0
./data_bmp/ACC\ETC\0_b43c93c42a2b5655bf534e25776caaaa_frame4.bmp
-114752760.0
./data_bmp/ACC\ETC\0_b43c93c42a2b5655bf534e25776caaaa_frame5.bmp
-132728660.0
./data_bmp/ACC\ETC\0_b43c93c42a2b5655bf534e25776caaaa_frame6.bmp
311794050.0
./data_bmp/ACC\ETC\10_5146221dee153df69a8479a101906808_frame1.bmp
./data_bmp/ACC\ETC\10_5146221dee153df69a8479a101906808_frame2.bmp
82707224.0
./data_bmp/ACC\ETC\11_0c4a50d1ee4205dde9a26413db75a9bb_frame1.bmp
./data_bmp/ACC\ETC\12_9c22aa80ae2922fac5cb341d54c64527_frame1.bmp
./data_bmp/ACC\ETC\12_9c22aa80ae2922fac5cb341d54c64527_frame2.bmp
-68881370.0
./data_bmp/ACC\ETC\12_9c22aa80ae2922fac5cb341d54c64527_frame3.bmp
489995420.0
./data_bmp/ACC\ETC\13_321e466b878aff0b2221

In [76]:
import pandas as pd
root_dir = "./data_bmp_sim/"

cnt = list()
total = 0
for (root, dirs, files) in os.walk(root_dir):
    print("# root : " + root)
    
    files = [i for i in files if not 'csv' in i]
    if len(files) > 0:
#         print("file: ",  len(files))
        cnt.append([root, len(files)])
        total+=len(files)
    else:
        cnt.append([root, 0])
#         index = 1
#         for file in files:
# #             print(os.path.join(root,file))
            
#             im = Image.open(os.path.join(root,file))
#             name = file.split('.')[0]
#             if not os.path.isdir(root.replace('data', 'data_bmp')):
#                 os.makedirs(root.replace('data', 'data_bmp'))
                
#             for frame in ImageSequence.Iterator(im):
#                 frame.save(root.replace('data', 'data_bmp')+'/'+name+"_frame%d.bmp" % index)
#                 index += 1

cnt.append(['total', total])
df = pd.DataFrame(cnt, columns=['root', 'count'])
print(df)
df.to_csv('./data_bmp/info_sim.csv', index=False)

# root : ./data_bmp_sim/
# root : ./data_bmp_sim/ACC
# root : ./data_bmp_sim/ACC\ETC
# root : ./data_bmp_sim/ACC\JEWELRY
# root : ./data_bmp_sim/ACC\SHOES & BAG
# root : ./data_bmp_sim/ACC\SOCKS
# root : ./data_bmp_sim/BIKINI
# root : ./data_bmp_sim/BOTTOM
# root : ./data_bmp_sim/BOTTOM\PANTS
# root : ./data_bmp_sim/BOTTOM\SKIRT
# root : ./data_bmp_sim/DRESS
# root : ./data_bmp_sim/LINGERIE
# root : ./data_bmp_sim/LINGERIE\LINGERIE
# root : ./data_bmp_sim/LINGERIE\PAJAMA
# root : ./data_bmp_sim/LINGERIE\SECRET BODY
# root : ./data_bmp_sim/MADE
# root : ./data_bmp_sim/OUTER
# root : ./data_bmp_sim/TOP
# root : ./data_bmp_sim/TOP\blouse
# root : ./data_bmp_sim/TOP\bra top
# root : ./data_bmp_sim/TOP\knit
# root : ./data_bmp_sim/TOP\sleeveless
# root : ./data_bmp_sim/TOP\tee
                                   root  count
0                       ./data_bmp_sim/      0
1                    ./data_bmp_sim/ACC      0
2                ./data_bmp_sim/ACC\ETC     65
3            ./data_bmp_sim/A

In [30]:
from PIL import Image, ImageSequence


index = 1
for frame in ImageSequence.Iterator(im):
    frame.save("frame%d.png" % index)
    index += 1